# Evaluación del Desempeño de modelos

In [1]:
import os

# Cambiar el directorio de trabajo
os.chdir('/Users/germanmejia/Desktop/MAESTRIA/2S/APRENDIZAJE AUTOMÁTICO/Módulo 1/Unidad 3')


# Etapa 1: Entendimiento de los Datos
Este paso es crucial para familiarizarse con los datos que vamos a utilizar. Durante esta etapa, realizaremos una exploración de los datos incluyendo el tipo de datos, valores faltantes y resumen estadístico.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Asumiendo que los datos están separados por espacios o un tabulador
df = pd.read_csv('Skin_NonSkin.txt', delimiter='\t', header=None, names=['B', 'G', 'R', 'Class'])

data = df  # Cambia 'df' por el nombre de tu DataFrame si es diferente

# Mostrar las primeras filas del DataFrame
print(data.head())

    B   G    R  Class
0  74  85  123      1
1  73  84  122      1
2  72  83  121      1
3  70  81  119      1
4  70  81  119      1


In [3]:
# Cuál es el número de registros y atributos?
shape = data.shape
print(f'Número de registros: {shape[0]}')
print(f'Número de atributos: {shape[1]}')

Número de registros: 245057
Número de atributos: 4


In [4]:
# Cuál es el tipo de los atributos?
print(data.dtypes)

B        int64
G        int64
R        int64
Class    int64
dtype: object


In [5]:
# Medida de centralidad y desviación para atributos numéricos:
print(data.describe())

                   B              G              R          Class
count  245057.000000  245057.000000  245057.000000  245057.000000
mean      125.065446     132.507327     123.177151       1.792461
std        62.255653      59.941197      72.562165       0.405546
min         0.000000       0.000000       0.000000       1.000000
25%        68.000000      87.000000      70.000000       2.000000
50%       139.000000     153.000000     128.000000       2.000000
75%       176.000000     177.000000     164.000000       2.000000
max       255.000000     255.000000     255.000000       2.000000


In [ ]:
# Matriz de correlación entre atributos numéricos
plt.figure(figsize=(10, 8))
# Asumiendo que las columnas B, G, R son las características numéricas
sns.heatmap(data[['B', 'G', 'R']].corr(), annot=True, fmt=".2f")
plt.show()

In [ ]:
# Identificación de datos faltantes
print(data.isnull().sum())

In [ ]:
# Máximo de datos faltantes en un mismo registro
print("Máximo de datos faltantes en un registro:", max(data.isnull().sum(axis=1)))


In [ ]:
# Identificar desbalance de clases en 'Class'
print(data['Class'].value_counts())

In [ ]:
# Seleccionar solo las columnas numéricas B, G, R
datos_numericos = data[['B', 'G', 'R']]
# Crear una figura y un eje para el gráfico
plt.figure(figsize=(15, 10))

# Crear los diagramas de cajas para B, G, R
sns.boxplot(data=datos_numericos, orient="h")

# Mostrar el gráfico
plt.show()

# Etapa 2: Preparación de los datos
De acuerdo a lo observado en la etapa 1, se define una secuencia de actividades que modifican los datos para eliminar las situaciones que puedan ser causa de fallo o deficiencia en el proceso de aprendizaje. 


Algunas acciones que pueden realizarse son:
- Eliminación o imputaciónde datos faltantes: En este caso no es necesario ya que no existe ningún dato faltante.
- Normalizar los valores de los atributos: En este caso tampoco serán necesarios por que los atributos hacen referencia a  valores de color RGB que tienen una escala de 0-255. 
- Convertir los atributos categóricos a escala numérica: Tampoco será necesario ya que están en formato numérico
- Balancear las clases eliminando registros de la clase mayoritaria o aumentando datos con diferentes técnicas de la clase minoritaria: En este caso, existe un  desbalance de clases. Sin embargo, primero se realizará el modelo con todos los datos para evaluar su rendimiento y luego se realizarán modificaciones para buscar el mejor modelo posible.
- Eliminar variables muy correlacionadas: En la matriz de correlaciones se observa que B y G tienen una alta correlación (0.86) y podrían dar la misma información. Por lo que se puede eliminar una de ellas. Sin embargo, al igual que lo anterior, primero evaluaremos el rendimiento del modelo con la base de datos original y luego realizaremos unos ajustes para buscar el mejor modelo posible.

**Rendimiento modelo conjunto de datos original**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Dividir el conjunto de datos en conjuntos de entrenamiento y test
X = data[['B', 'G', 'R']]  # Características
y = data['Class']  # Etiqueta

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Entrenamiento del modelo de clasificación por regresión logística
logisticRegr = LogisticRegression(solver="lbfgs", max_iter=500, tol=0.01)
logisticRegr.fit(X_train, y_train)

# Aplicación del modelo construido a los datos de test
predictions = logisticRegr.predict(X_test)

# Cálculo del accuracy para evaluar el desempeño del modelo sobre los datos de test
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy del modelo: {accuracy}')

# Reporte de clasificación
report = classification_report(y_test, predictions)
print(report)


**Rendimiento modelo con submuestreo**

In [ ]:
from sklearn.utils import resample

# Crear una copia del DataFrame para mantener el original intacto
data_subsampled = data.copy()

# Separar el conjunto de datos por clase
data_class_1 = data_subsampled[data_subsampled.Class == 1]
data_class_2 = data_subsampled[data_subsampled.Class == 2]

# Submuestrear la clase mayoritaria sin reemplazo para igualar el número de la clase minoritaria
data_class_2_subsampled = resample(data_class_2, 
                                   replace=False, 
                                   n_samples=len(data_class_1), 
                                   random_state=42)

# Combinar la clase minoritaria con la clase mayoritaria submuestreada
data_balanced_subsampled = pd.concat([data_class_1, data_class_2_subsampled])

# Mezclar los datos para evitar cualquier tipo de orden
data_balanced_subsampled = data_balanced_subsampled.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# Dividir el conjunto de datos balanceado con submuestreo en conjuntos de entrenamiento y test
X_subsampled = data_balanced_subsampled[['B', 'G', 'R']]
y_subsampled = data_balanced_subsampled['Class']

X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(
    X_subsampled, y_subsampled, test_size=0.30, random_state=42)

# Entrenamiento del modelo con el conjunto submuestreado
logisticRegr_subsampled = LogisticRegression(solver="lbfgs", max_iter=500)
logisticRegr_subsampled.fit(X_train_sub, y_train_sub)

# Predicciones con el conjunto de prueba
predictions_subsampled = logisticRegr_subsampled.predict(X_test_sub)

# Evaluación del modelo submuestreado
accuracy_subsampled = accuracy_score(y_test_sub, predictions_subsampled)
print(f'Accuracy del modelo con submuestreo: {accuracy_subsampled}')
print(classification_report(y_test_sub, predictions_subsampled))


**Rendimiento modelo con sobrebmuestreo usando SMOTE**

La técnica de SMOTE (Synthetic Minority Over-sampling Technique) es un método de balanceo de datos utilizado en machine learning para aumentar la cantidad de datos de las clases minoritarias. Funciona generando datos sintéticos, es decir, muestras artificiales, a partir de las existentes. SMOTE selecciona muestras aleatorias de la clase minoritaria, encuentra sus k vecinos más cercanos (usualmente en el espacio de características), y luego crea nuevas muestras que son combinaciones lineales de la muestra seleccionada y sus vecinos. Esto ayuda a evitar el sobreajuste que puede ocurrir con el sobremuestreo simple replicando datos, y mejora el rendimiento de los modelos al tener un conjunto de datos más equilibrado.

In [ ]:
# Crear una copia del DataFrame para mantener el original intacto
data_oversampled = data.copy()

# Sobremuestrear la clase minoritaria con reemplazo para igualar el número de la clase mayoritaria
data_class_1_oversampled = resample(data_class_1, 
                                    replace=True, 
                                    n_samples=len(data_class_2), 
                                    random_state=42)

# Combinar la clase mayoritaria con la clase minoritaria sobremuestreada
data_balanced_oversampled = pd.concat([data_class_2, data_class_1_oversampled])

# Mezclar los datos para evitar cualquier tipo de orden
data_balanced_oversampled = data_balanced_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# !pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

# Crear una copia del DataFrame para mantener el original intacto
X = data[['B', 'G', 'R']].copy()
y = data['Class'].copy()

# Inicializar SMOTE y sobremuestrear la clase minoritaria
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Convertir los resultados en un DataFrame
data_balanced_smote = pd.DataFrame(X_smote, columns=['B', 'G', 'R'])
data_balanced_smote['Class'] = y_smote

# Mezclar los datos para evitar cualquier tipo de orden
data_balanced_smote = data_balanced_smote.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# Dividir el conjunto de datos balanceado con sobremuestreo en conjuntos de entrenamiento y test
X_oversampled = data_balanced_oversampled[['B', 'G', 'R']]
y_oversampled = data_balanced_oversampled['Class']

X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(
    X_oversampled, y_oversampled, test_size=0.30, random_state=42)

# Entrenamiento del modelo con el conjunto sobremuestreado
logisticRegr_oversampled = LogisticRegression(solver="lbfgs", max_iter=500)
logisticRegr_oversampled.fit(X_train_over, y_train_over)

# Predicciones con el conjunto de prueba
predictions_oversampled = logisticRegr_oversampled.predict(X_test_over)

# Evaluación del modelo sobremuestreado
accuracy_oversampled = accuracy_score(y_test_over, predictions_oversampled)
print(f'Accuracy del modelo con sobremuestreo: {accuracy_oversampled}')
print(classification_report(y_test_over, predictions_oversampled))


**Rendimiento del modelo eliminando variables muy correlacionadas**

La B y la G tienen una correlación positiva muy alta. Esto nos habla de que la información que aportan es muy similar y podría eliminarse una de ellas para reducir la dimensionalidad.

In [ ]:
# Crear una nueva copia del DataFrame sin la columna 'G'
data_without_G = data.copy().drop('G', axis=1)
# Dividir el nuevo conjunto de datos en características y etiquetas
X_no_G = data_without_G.drop('Class', axis=1)
y_no_G = data_without_G['Class']

# Dividir en conjunto de entrenamiento y prueba
X_train_no_G, X_test_no_G, y_train_no_G, y_test_no_G = train_test_split(
    X_no_G, y_no_G, test_size=0.30, random_state=42)

# Entrenamiento del modelo
logisticRegr_no_G = LogisticRegression(solver="lbfgs", max_iter=500)
logisticRegr_no_G.fit(X_train_no_G, y_train_no_G)

# Predicciones
predictions_no_G = logisticRegr_no_G.predict(X_test_no_G)

# Evaluación del modelo
accuracy_no_G = accuracy_score(y_test_no_G, predictions_no_G)
print(f'Accuracy del modelo sin la variable G: {accuracy_no_G}')
print(classification_report(y_test_no_G, predictions_no_G))


**Rendimiento modelo con submuestreo y eliminación de variables muy correlacionadas**

In [ ]:
# Paso 2: Eliminar la variable 'G' del conjunto de datos submuestreado
data_subsampled_no_G = data_subsampled.drop('G', axis=1)

# Paso 3: Dividir el conjunto de datos en entrenamiento y prueba
X_subsampled_no_G = data_subsampled_no_G.drop('Class', axis=1)
y_subsampled_no_G = data_subsampled_no_G['Class']

X_train_sub_no_G, X_test_sub_no_G, y_train_sub_no_G, y_test_sub_no_G = train_test_split(
    X_subsampled_no_G, y_subsampled_no_G, test_size=0.30, random_state=42)

# Paso 4: Entrenamiento del modelo de regresión logística
logisticRegr_sub_no_G = LogisticRegression(solver="lbfgs", max_iter=500)
logisticRegr_sub_no_G.fit(X_train_sub_no_G, y_train_sub_no_G)

# Paso 5: Predicciones con el conjunto de prueba
predictions_sub_no_G = logisticRegr_sub_no_G.predict(X_test_sub_no_G)

# Paso 6: Evaluación del modelo
accuracy_sub_no_G = accuracy_score(y_test_sub_no_G, predictions_sub_no_G)
print(f'Accuracy del modelo con submuestreo y sin la variable G: {accuracy_sub_no_G}')
print(classification_report(y_test_sub_no_G, predictions_sub_no_G))

**Comparación modelos**

In [ ]:
import pandas as pd

# Resultados de los modelos
resultados = {
    "Modelo": [
        "Modelo Original",
        "Modelo con Submuestreo",
        "Modelo con Sobremuestreo",
        "Modelo sin 'G'",
        "Modelo con Submuestreo y sin 'G'"
    ],
    "Accuracy": [
        0.9190,
        0.9436,
        0.9409,
        0.9170,
        0.9170
    ],
    "Precision (Macro Avg)": [
        0.87,
        0.94,
        0.94,
        0.87,
        0.87
    ],
    "Recall (Macro Avg)": [
        0.88,
        0.94,
        0.94,
        0.88,
        0.88
    ],
    "F1-Score (Macro Avg)": [
        0.88,
        0.94,
        0.94,
        0.88,
        0.88
    ],
    "Precision (Weighted Avg)": [
        0.92,
        0.94,
        0.94,
        0.92,
        0.92
    ],
    "Recall (Weighted Avg)": [
        0.92,
        0.94,
        0.94,
        0.92,
        0.92
    ],
    "F1-Score (Weighted Avg)": [
        0.92,
        0.94,
        0.94,
        0.92,
        0.92
    ]
}

# Crear un DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)

# Mostrar la tabla
df_resultados


# Etapa 3: Evaluación del modelo.

Para la evaluación escogimos el modelo con mejor accuracy. es decir el modelo obtenido con el submuestreo. En esta etapa realizamos la evaluación con base a dos aspectos
1. Validación Cruzada con N-folds (N=10).
2. Random sub sampling 70/30con 10 repeticiones.

A cada uno le calculamos lo siguiente: Matriz de confusión,Accuracy,Sensitivity o Recall, Specificity, Precision.
F1-score.

**Validación cruzada**

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
import numpy as np

# Definir el número de divisiones para la validación cruzada
n_splits = 10

# Crear la instancia de StratifiedKFold
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Asumiendo que 'X' e 'y' ya están definidos, así como 'logisticRegr'

# Inicializar listas para guardar las métricas de cada iteración
conf_matrices = []
accuracies = []
recalls = []
precisions = []
f1_scores = []
specificities = []

# Ejecutar la validación cruzada
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    logisticRegr.fit(X_train, y_train)
    predictions = logisticRegr.predict(X_test)
    
    # Obtener y almacenar la matriz de confusión
    cm = confusion_matrix(y_test, predictions)
    TN = cm[0][0]
    FP = cm[0][1]
    
    # Calcular la especificidad y otros indicadores
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    specificities.append(specificity)
    accuracies.append(accuracy_score(y_test, predictions))
    recalls.append(recall_score(y_test, predictions, average='macro'))
    precisions.append(precision_score(y_test, predictions, average='macro'))
    f1_scores.append(f1_score(y_test, predictions, average='macro'))

# Calcular y mostrar promedios de las métricas
avg_specificity = np.mean(specificities)
print("Promedio Matriz de Confusión:\n", np.mean(conf_matrices, axis=0))
print("Promedio Accuracy:", np.mean(accuracies))
print("Promedio Recall:", np.mean(recalls))
print("Promedio Precision:", np.mean(precisions))
print("Promedio F1-Score:", np.mean(f1_scores))
print("Promedio Especificidad:", avg_specificity)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Tu matriz de confusión promedio obtenida de la validación cruzada
conf_matrix = np.array([[4188.1, 897.8],
                        [1092.5, 18327.3]])

# Normalizar la matriz de confusión manualmente para la visualización de la matriz normalizada
conf_matrix_normalized = conf_matrix / conf_matrix.sum(axis=1).reshape(-1, 1)

# Etiquetas de clases para tu caso específico
class_names = ['Clase 1', 'Clase 2']  # Ajusta según tus clases

# Configuración de títulos para las matrices de confusión
title_options = [
    ("Matriz de Confusión sin Normalización", None),
    ("Matriz de Confusión Normalizada", conf_matrix_normalized),
]

for title, matrix in title_options:
    fig, ax = plt.subplots(figsize=(6, 6))
    if matrix is None:
        disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,
                                      display_labels=class_names)
    else:
        disp = ConfusionMatrixDisplay(confusion_matrix=matrix,
                                      display_labels=class_names)
    disp.plot(include_values=True,
              cmap=plt.cm.Blues, ax=ax, xticks_rotation='horizontal',
              values_format='.2f' if title.find("Normalizada") > -1 else '.0f')
    ax.set_title(title)

plt.show()


**Random sub sampling 70/30con 10 repeticiones**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
import numpy as np

# Asumiendo que X, y, y logisticRegr ya están definidos

# Inicializar listas para guardar las métricas de cada repetición, incluida la especificidad
conf_matrices_rs = []
accuracies_rs = []
recalls_rs = []
precisions_rs = []
f1_scores_rs = []
specificities_rs = []

# Realizar 10 repeticiones de random subsampling
for i in range(10):
    X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(X, y, test_size=0.30, random_state=i)
    
    logisticRegr.fit(X_train_rs, y_train_rs)
    predictions_rs = logisticRegr.predict(X_test_rs)
    
    # Obtener la matriz de confusión
    cm = confusion_matrix(y_test_rs, predictions_rs)
    conf_matrices_rs.append(cm)
    
    # Calcular TN y FP para la especificidad
    TN = cm[0][0]
    FP = cm[0][1]
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    specificities_rs.append(specificity)
    
    # Calcular otras métricas
    accuracies_rs.append(accuracy_score(y_test_rs, predictions_rs))
    recalls_rs.append(recall_score(y_test_rs, predictions_rs, average='macro'))
    precisions_rs.append(precision_score(y_test_rs, predictions_rs, average='macro'))
    f1_scores_rs.append(f1_score(y_test_rs, predictions_rs, average='macro'))

# Calcular promedios de las métricas, incluyendo la especificidad
avg_specificity_rs = np.mean(specificities_rs)

# Imprimir resultados
print("Random Subsampling - Promedio Matriz de Confusión:\n", np.mean(conf_matrices_rs, axis=0))
print("Promedio Accuracy:", np.mean(accuracies_rs))
print("Promedio Recall:", np.mean(recalls_rs))
print("Promedio Precision:", np.mean(precisions_rs))
print("Promedio F1-Score:", np.mean(f1_scores_rs))
print("Promedio Especificidad:", avg_specificity_rs)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# Actualizar la matriz de confusión promedio obtenida del random subsampling
conf_matrix = np.array([[12572.1, 2696.1],
                        [3281.3, 54968.5]])

# Normalizar la matriz de confusión manualmente para la visualización de la matriz normalizada
conf_matrix_normalized = conf_matrix / conf_matrix.sum(axis=1, keepdims=True)

# Etiquetas de clases para tu caso específico
class_names = ['Clase 1', 'Clase 2']

# Configuración de títulos para las matrices de confusión
title_options = [
    ("Matriz de Confusión sin Normalización", conf_matrix),
    ("Matriz de Confusión Normalizada", conf_matrix_normalized),
]

for title, matrix in title_options:
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=matrix,
                                  display_labels=class_names)
    disp.plot(include_values=True,
              cmap=plt.cm.Blues, ax=ax, xticks_rotation='horizontal',
              values_format='.2f' if title.find("Normalizada") > -1 else '.0f')
    ax.set_title(title)

plt.show()


# Análisis y conclusión

En este proyecto, hemos abordado el desafío de clasificar píxeles de imágenes en dos categorías: piel y no piel, utilizando un conjunto de datos que comprende valores de color en el espacio RGB. Nuestro enfoque metodológico se centró en explorar diferentes estrategias para mejorar el rendimiento de un modelo de regresión logística, dada la presencia de desequilibrio de clases y la correlación entre algunas características. 

Inicialmente, llevamos a cabo un análisis exploratorio para comprender la naturaleza de nuestros datos, confirmar la ausencia de valores faltantes y analizar la correlación entre las características. Observamos una alta correlación entre los canales de color azul (B) y verde (G), y un notable desequilibrio de clases, con una predominancia de píxeles no correspondientes a piel.

Para abordar el desequilibrio de clases, implementamos dos estrategias: submuestreo de la clase mayoritaria y sobremuestreo de la clase minoritaria utilizando la técnica SMOTE. Además, experimentamos eliminando la característica 'G' para reducir la multicolinealidad.Posteriormente se evaluó el modelo con mejor rendimiento, es decir el modelo obtenido con el submuestreo mediante dos técnicas: validación cruzada N-folds (N=10) y random subsampling (70/30) con 10 repeticiones. Las métricas de evaluación incluyeron accuracy, precisión (macro y ponderada), recall (macro y ponderada), F1-score (macro y ponderada) y especificidad.

La validación cruzada N-folds y el random subsampling tuvieron resultados muy similares. Esta consistencia sugiere que el modelo es estable y generaliza bien a nuevos datos, manteniendo un rendimiento sólido en todas las métricas clave.

**Fin**